In [29]:
!pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 1.9 MB 4.6 MB/s            
     |████████████████████████████████| 168 kB 105.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 182 kB 79.1 MB/s            
     |████████████████████████████████| 62 kB 2.7 MB/s             
     |████████████████████████████████| 140 kB 75.2 MB/s            
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=23348fe6a424decbe9e8043d38f208adfc94f6bf464cbec9ec52484c9bb4db86
  Stored in directory: /aiffel/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtool

In [ ]:
# mecab 설치
# !curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [1]:
!pip install kogpt2_transformers
!pip install rouge

In [3]:
!pip install pandas==1.3.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 87.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.1/498.1 kB 52.8 MB/s eta 0:00:00


In [17]:
!pip install matplotlib==3.4.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 72.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 86.2 MB/s eta 0:00:00


In [1]:
# 라이브러리 불러오기

import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset
from kogpt2_transformers import get_kogpt2_model, get_kogpt2_tokenizer
import csv
import pandas as pd
#from datasets import Dataset
from rouge import Rouge
import re
import os

In [49]:
# Category별 DataFrame을 출력 (train)
train = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/train.tsv', delimiter = '\t')
Category = train['Category'].unique()
data_path = '/aiffel/aiffel/aiffelthon/csv/'
def train_categori_ext(data):
    df = pd.DataFrame()
    for c in Category:
        df = pd.concat([df, data[data['Category'] == c].iloc[0:int(len(data[data['Category'] == c])*0.05)]], axis = 0)
        df.reset_index(inplace=True, drop=True)
        df.to_csv(data_path+'train_5%.tsv', sep = '\t', encoding = 'utf-8', index = False)
    return df

# 데이터 파일 생성

train_categori_ext(train)

,Id,Text,Summary,Category
0,fd321028-d5b4-55f7-9e20-2eaa262f9154,"['그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정할까?', '우리 비행...","비행기 표 가격에 대해 이야기하며, 특가 이벤트를 기다리고 있다.",상거래(쇼핑)
1,c51be2e4-c8d0-5cea-b1ae-cde1fe8f8ab6,"['Kf마스크만 5부제 하는거지?', '응. 면마스크는 아무때나 사도될껀?', '면...",비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다.,상거래(쇼핑)
2,e90e721f-00d1-5114-aa5d-5f1061472a29,['아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 그냥그런것같애'...,케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...,상거래(쇼핑)
3,b215f3a2-d647-59f9-8410-1274ee5edd97,"['칫솔사야하는데 쓱으로 살까?', '뭘 칫솔사는것까지 물어보시남ㅋㅋㅋ', '아 그...",칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계(쓱) 가자고 했다.,상거래(쇼핑)
4,0bda61b6-1396-5a2a-a049-0b4035e40d59,"['잠도안오네ㅐ얼릉 고구마츄 먹고싶단', '그게 그렇게 맛있었어??? 아주 여보 빼...",잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다.,상거래(쇼핑)
...,...,...,...,...
13989,94a9b0e9-5a84-5739-9851-f5bae80d936d,"['옷 오늘간대', '오 드디어 저녁에 입어야지 #@이모티콘#', '예상저녁7시~9...",배송 예상 시간이 7-9시인데 옷을 입고 나오라고 했다.,행사
13990,5eed65c3-f9a9-5736-a400-ca79f43516fe,"['함안올끼가?', '#@이름# 데꼬? ㅎ 일주일요양가까', '어', '이번주말고 ...",이번 주에 비 소식이 있어 다음 주에 가기로 한다.,행사
13991,6505168e-cccb-5bdf-8cc9-19b488f6b8cf,"['언니 안뇽! 잘지냉?', '#@이름#아!!! 히힛 난 잘지내! 너는 잘지내구이쪄...",8월 첫째 주가 휴가라서 시간이 되면 만나자고 했다.,행사
13992,3dd63c1d-22d3-5720-b30e-d9d6fd81d6f5,"['그럼 거기그때 진흥아파트??', '네넨 진흥아파트에요~', '오키', '올라오시...",1시 반 넘어서 만나기로 하고 진흥아파트 동 호수를 알려주었다.,행사


In [50]:
# Category별 DataFrame을 출력
val = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/val.tsv', delimiter = '\t')
Category = val['Category'].unique()
data_path = '/aiffel/aiffel/aiffelthon/csv/'
def val_categori_ext(data):
    df = pd.DataFrame()
    for c in Category:
        df = pd.concat([df, data[data['Category'] == c].iloc[0:int(len(data[data['Category'] == c])*0.05)]], axis = 0)
        df.reset_index(inplace=True, drop=True)
        df.to_csv(data_path+'val_5%.tsv', sep = '\t', encoding = 'utf-8', index = False)
    return df

# 데이터 파일 생성

val_categori_ext(val)

,Id,Text,Summary,Category
0,9effd65e-e18f-5510-8866-d6a677314c9c,"['웅', '영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산갈때도 숙...",팀장님이 출장 가서 머물 숙소를 계속해서 더 싼 데로 하게 한다고 이야기하고 있다.,일과 직업
1,49b9f9b7-a79d-5283-9553-95b8f156ff90,"['너는 잘가라....회사.... 선택 잘해..', '알겠어 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 많...",이제 이력서를 쓰고 영어도 해야 한다고 해서 첫 회사를 잘 들어가라고 했다.,일과 직업
2,df6b6bac-648b-5d32-86a3-9b8b215e5f95,"['느낌상 대통령까지는 아니고 오시면 여사님정도오시지않을까 ㅋㅋㅋ', '그러면서',...",느낌상 대통령까지는 아니고 오시면 여사님 정도 오시지 않을까라며 이에 대해 이야기하...,일과 직업
3,3043293f-7bb5-5edb-9b1f-85120e305c3d,"['숨만수이ㅓ도 숨만쉬어도 100 이내', '한달안에 일 무조건 해야대', '아 딱...",한 달 안에 무조건 일을 시작해서 돈을 벌어야 한다.,일과 직업
4,2b9b9c39-2721-50e6-af3e-e5c00138f43d,"['목요일은 외근이구 금요일은 출장!!!', '금요일이 당진이양?', '아닝아닝 1...",목요일에 외근이고 금요일에 출장인데 당진은 10일에 간다.,일과 직업
...,...,...,...,...
1741,b03b9ad0-62fa-54a5-b552-69049bd642ee,"['엄마 점심식사 뭐먹었어요??', '우린 중국집 시켜먹었다', '오오 짜장면 탕수...","엄마에게 오늘 점심 뭐 먹었냐고 묻고 엄마가 중국집에서 볶음밥, 잡채밥 시켜 먹었다...",식음료
1742,ec83f40a-3c01-581d-962c-80fd62a5c888,"['매운거', '담에 점심 엽떡', '자신있음', '엽떡 먹는 사람', '오 저색기...",엽떡(엽기 떡볶이) 제일 매운맛을 3명에서 도전하기로 했다.,식음료
1743,2b8dad32-cb6a-5d2a-89c4-06ba5f8dc8f8,"['아 술마시고싶다 취할줄알았는데 ㅋㅋㅋㅋㅋㅋ', '누룩오자 누룩 군고구마막걸리 존...",군고구마막걸리가 맛있다고 하니 빨리 마셔보고 싶다고 한다.,식음료
1744,bd803ef5-c166-54ad-8d96-ac73fb055b3d,"['종촌동에서 뭘 주워먹어야겠다... 어른들 저녁먹고 가신대 뭐사먹지,.....',...",어른들이 저녁 먹고 가신다고 해서 뭘 사 먹을까 하니 근처에 국수나무가 맛있다고 한다.,식음료


In [13]:
# torch cash 초기화

import gc
gc.collect()
torch.cuda.empty_cache()

Model : KoGPT-2

Parametors size : 

[How to use GPT2LMHeadModel for conditional generation](https://github.com/huggingface/transformers/issues/970)

Data NAME : Ai_hub 대화문 요약 Data Set

In [2]:
# 데이터 전처리

def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (.) 제거
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ.,?!]+[/ㄱ-ㅎㅏ-ㅣ.,?!]', '', sentence) # 여러개 자음, 모음, 구두점 제거
    sentence = re.sub("[^가-힣a-z0-9-.,!?#@]", " ", sentence) # 지정한 문자 제외 공백으로 전환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거

    return sentence

In [11]:
text = '#@주소# 여기로 배송해줘'
preprocess_sentence(text)

'#@주소# 여기로 배송해줘'

In [3]:
# 데이터 로드 및 전처리와 메타데이터 적용 

def data_load(data_path = '/home/rlfkr6974/Download/csv/train_5%.tsv'):
    data = []
    train_df = pd.read_csv(data_path, delimiter = '\t')
    #train_df = train_df.iloc[::2, :] # 짝수 row data만 추출한다.
    #train_df.reset_index(drop=True, inplace = True) # 인덱스 초기화
    
    # 전처리 적용
    clean_text = []
    
    for s in tqdm(train_df['Text']):
        clean_text.append(preprocess_sentence(s))
                          
    train_df['Text'] = clean_text
    
    clean_headlines = []
                          
    for s in tqdm(train_df['Summary']):
        clean_headlines.append(preprocess_sentence(s))
        
    train_df['Summary'] = clean_headlines 
    
    # 메타데이터 적용
    def metadata_train(train_df): 
        meta = '#'+train_df['Category']+'#'
        train_df['Text'] = meta + train_df['Text']
        return train_df['Text']
    
    metadata_train(train_df)
    
    # Series > list
    train_df['Text'].tolist()
    train_df['Summary'].tolist()
    
    return train_df['Text'], train_df['Summary']

def val_load(data_path = '/home/rlfkr6974/Download/csv/val_5%.tsv'):
    val_df = pd.read_csv(data_path, delimiter = '\t')
    #val_df = val_df.iloc[::2, :]
    #val_df.reset_index(drop=True, inplace = True)
    

    # 전처리 적용
    clean_text_val = []
                          
    for s in tqdm(val_df['Text']):
        clean_text_val.append(preprocess_sentence(s))
    
    val_df['Text'] = clean_text_val
    
    clean_headlines_val = []
                          
    for s in tqdm(val_df['Summary']):
        clean_headlines_val.append(preprocess_sentence(s))
        
    val_df['Summary'] = clean_headlines_val 
    
    # 메타데이터 적용
    def metadata_val(val_df): 
        meta = '#'+val_df['Category']+'#'
        val_df['Text'] = meta + val_df['Text']
        return val_df['Text']
    
    metadata_val(val_df)
                          
    # Series > list
    val_df['Text'].tolist()
    val_df['Summary'].tolist()
    
    return val_df['Text'], val_df['Summary']

In [16]:
data_1 = data_load()

100%|██████████| 55993/55993 [00:00<00:00, 96673.03it/s]


In [17]:
data_1

(0        #상거래(쇼핑)# 그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정할까? ...
 1        #상거래(쇼핑)# kf마스크만 5부제 하는거지? , 응. 면마스크는 아무때나 사도될...
 2        #상거래(쇼핑)# 아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 ...
 3        #상거래(쇼핑)# 칫솔사야하는데 쓱으로 살까? , 뭘 칫솔사는것까지 물어보시남 , ...
 4        #상거래(쇼핑)# 잠도안오네 얼릉 고구마츄 먹고싶단 , 그게 그렇게 맛있었어 아주 ...
                                ...                        
 55988    #행사# 일본으로 낚시가 , 그건또첨인데 맛있는거만 먹으러다녔었지 낚시는 한번두 ,...
 55989    #행사# 이모부생신27일 그래서 투썸케이크 방금전화로예약함 , 이름 이생일은 28일...
 55990    #행사# 이게 무슨일 십삼만원 확인해보니까 다낭가는게 4만 돌아오는게 6만이네 취소...
 55991    #행사# 병원 갔다가 칸다소바나 갈까 혼자 점심은 어디서 좋을까 , 칸다소바 가라 ...
 55992    #행사# 겨론식끝나쪄용 , 이열 오늘 언니결혼이였꾸나꺅 축하드립니다 오나애 화장이랑...
 Name: Text, Length: 55993, dtype: object,
 0                     비행기 표 가격에 대해 이야기하며, 특가 이벤트를 기다리고 있다.
 1                      비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다.
 2        케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...
 3                     칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계 가자고 했다.
 4                        잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한

In [139]:
data_2 = val_load()

100%|██████████| 17502/17502 [00:00<00:00, 95907.57it/s]


In [140]:
type(data_1)

tuple

In [142]:
data_2

(0        #일과 직업# 웅 , 영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산...
 1        #일과 직업# 느낌상 대통령까지는 아니고 오시면 여사님정도오시지않을까 , 그러면서 ...
 2        #일과 직업# 목요일은 외근이구 금요일은 출장 , 금요일이 당진이양? , 아닝아닝 ...
 3        #일과 직업# 메일 주소 알려주세여 제가 점심먹고와서 보낼게여 , 넵넵 시스템 사진...
 4        #일과 직업# 나 크리스마스떄도 일한다는데 시급 1.5배 맞나요 , 어 1.5배 요...
                                ...                        
 17497    #식음료# 마카롱은 , 우리 들어오자마자 우걱우걱 , 2개 정도 먹어야함 후 자제하...
 17498    #식음료# 저번에 스벅에서 바움쿠헨 먹었거둥 나원래 바움쿠헨 좋아하긴하거든 근데 저...
 17499    #식음료# 배고프닭 5시반에 시키면 5시 40분에 주문 , 언니씻고 뭐하면 , 맵구...
 17500    #식음료# 닭갈비 맛있었다 맛나맛나 그티웅 , 아깐 먹기 싫다더니 , 역시 너의 선...
 17501    #식음료# 아! 본죽에 신메뉴 나온 거 알아 , 오 신메뉴 나온 걸 왜 못봤지 ? ...
 Name: Text, Length: 17502, dtype: object,
 0           팀장님이 출장 가서 머물 숙소를 계속해서 더 싼 데로 하게 한다고 이야기하고 있다.
 1        느낌상 대통령까지는 아니고 오시면 여사님 정도 오시지 않을까라며 이에 대해 이야기하...
 2                         목요일에 외근이고 금요일에 출장인데 당진은 10일에 간다.
 3           점심을 먹고 와서 알려준 메일 주소로 롯데 원티비 큐에이 담당자에게 메일을 보냈다.
 4        근로계약서를 써야한다고 하며 크리스마스 때 일하는 것의 시급을 1.5배해달라고 할 .

### hyparam은 Code를 가져올 때 설정을 유지중입니다. GPT-2  학습이 제대로 이루어진게 어제부터라서 수정하지는 못했습니다.

### 그래도 수정을 한 부분은 문장을 생성할 때 eos_token, pad_token이 생성되어 이 부분을 출력하지 않도록 bad_words_index를 사용했습니다.

In [4]:
# 토크나이저 및 모델 선언

tokenizer = get_kogpt2_tokenizer()
model = get_kogpt2_model()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [8]:
text = "@주소# 여기로 와라"

tokenizer.tokenize(text)

['▁@', '주소', '#', '여', '기로', '▁와', '라']

In [5]:
# 스페셜 토큰 적용
#added_token_num = tokenizer
#added_token_num = tokenizer.add_tokens([''])

special_words = [
            "#@주소#", "#@이모티콘#", "#@이름#", "#@URL#", "#@소속#",
            "#@기타#", "#@전번#", "#@계정#", "#@url#", "#@번호#", "#@금융#", "#@신원#",
            "#@장소#", "#@시스템#동영상#", "#@시스템#기타#", "#@시스템#검색#",
            "#@시스템#지도#", "#@시스템#사진#", "#@시스템#삭제#", "#@시스템#파일#", "#@시스템#송금#", "#@시스템#",
            '#상거래(쇼핑)#','#개인 및 관계#', '#식음료#', '#시사/교육#', '#미용과 건강#', '#일과 직업#', '#여가 생활#', 
            '#주거와 생활#', '#행사#']

tokenizer.add_special_tokens({"additional_special_tokens": special_words})
#tokenizer.add_special_tokens

# 임베딩 바꾸기
#model.resize_token_embeddings(len(tokenizer))
len(tokenizer)

50155

In [6]:
# Model 제작

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from kogpt2_transformers import get_kogpt2_model


class AbstractiveKoGPT2(nn.Module):
    def __init__(self):
        super(AbstractiveKoGPT2, self).__init__()
        self.kogpt2 = get_kogpt2_model()

    def generate(self,
                 input_ids,
                 #do_sample=True,
                 max_length= 128,
                 num_beams=5,
                 length_penalty = 1.2,
                 #top_p=0.92, # top_p, k는 code를 가져올 때 설정을 유지하였고 보내주신 텍스트 생성
                 #top_k=50,
                 temperature= 0.6, # 0.6 or 0.8 
                 no_repeat_ngram_size = None, # 변경 시도해보기
                 num_return_sequences=3,
                 early_stopping=False,
                 bad_words_ids = [1, 3]
                 ):
        return self.kogpt2.generate(input_ids,
                                     #do_sample = do_sample,
                                     max_length = max_length,
                                     num_beams = num_beams, # 더 많은 글 작성 예시 생성
                                     length_penalty = length_penalty,
                                     #top_p = top_p, # 글의 표현 범위 조절
                                     #top_k = top_k, # 글의 표현 범위 조절
                                     temperature = temperature, # 글의 창의성 조절
                                     no_repeat_ngram_size= no_repeat_ngram_size,
                                     num_return_sequences = num_return_sequences,
                                     early_stopping = early_stopping,
                                     eos_token_id = 1,
                                     pad_token_id = 3,
                                    )

    def forward(self, input, labels = None):
        if labels is not None:
            outputs = self.kogpt2(input, labels=labels)
        else:
            outputs = self.kogpt2(input)

        return outputs

### 스페셜 토큰은 당시 대회 1등을 수상한 팀이 사전에 파악해둔 라벨링 텍스트를 토대로 가져왔으며, 정규표현식을 사용해 해당 텍스트가 데이터에 포함된 것을 확인했습니다.

In [7]:
# 학습용 데이터 세트 제작

class AbstrativeDataset(Dataset):
    def __init__(self,
                 #n_ctx = 1024, 
                 n_ctx = 256, 
                 texts_max_length = 128,
                 summary_max_length = 64,
                 device = 'cpu'
                 ):
        
        self.data =[]
        self.device = device
        self.tokenizer = get_kogpt2_tokenizer()

        bos_token_id = [self.tokenizer.bos_token_id] # , 0
        eos_token_id = [self.tokenizer.eos_token_id] # , 1
        pad_token_id = [self.tokenizer.pad_token_id] # , 3
        
#         special_words = [
#         "#@주소#", "#@이모티콘#", "#@이름#", "#@URL#", "#@소속#",
#         "#@기타#", "#@전번#", "#@계정#", "#@url#", "#@번호#", "#@금융#", "#@신원#",
#         "#@장소#", "#@시스템#사진#", "#@시스템#동영상#", "#@시스템#기타#", "#@시스템#검색#",
#         "#@시스템#지도#", "#@시스템#삭제#", "#@시스템#파일#", "#@시스템#송금#", "#@시스템#",]
        
        
#         self.tokenizer.add_special_tokens({"additional_special_tokens": special_words}) 
#         # 스페셜 토큰은 적용

        datas = data_load()
        
        for texts, text_sum in tqdm(zip(datas[0],datas[1])):
            text = texts
            abstractive_summary = text_sum

#             tmp_str =''
#             for article in articles:
#                 tmp_str += article

            # encode
            # truncate, if string exceed max length
            enc_tmp_str = self.tokenizer.encode(text, truncation= True, max_length=texts_max_length, add_special_tokens=False)
            enc_abstractive_summary = self.tokenizer.encode(abstractive_summary, truncation= True, max_length=summary_max_length)
            
            #  요약할 문장  요약된 문장 
            index_of_words = bos_token_id + enc_tmp_str+ eos_token_id + enc_abstractive_summary + eos_token_id
            pad_token_len = n_ctx - len(index_of_words)
            index_of_words += pad_token_id * pad_token_len
            #print(f'max: {max(index_of_words)}, min: {min(index_of_words)}')
            
            # RuntimeError: CUDA error: device-side assert triggered
            if max(index_of_words)<50000:
                self.data.append(torch.tensor(index_of_words).to(device))

    def __len__(self):
        return len(self.data)

    def __getitem__(self,index):
        item = self.data[index]
        return item

In [9]:
data_3 = AbstrativeDataset()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
100%|█████████████████████████████████████████████████████████████████| 13994/13994 [00:00<00:00, 86065.33it/s]
13994it [00:04, 2901.80it/s]


In [10]:
# 데이터 셋이 제대로 만들어졌는지 확인 완료

len(data_3)

13994

In [67]:
# 데이터 셋이 제대로 만들어지지 않은 경우

len(dataset)

0

In [8]:
# 검증용 데이터 세트 제작
class EvalAbstrativeDataset(Dataset):
    def __init__(self,
                 device,
                 tokenizer,
                 #n_ctx = 1024,
                 n_ctx = 256,
                 data_path = '/home/rlfkr6974/Download/csv/val_5%.tsv',
                 texts_max_length = 128,
                 summary_max_length = 64,
                 ):
        #self.data = pd.DataFrame()
        self.data =[]
        self.tokenizer = tokenizer
        print(device)
        
        bos_token_id = [self.tokenizer.bos_token_id] # 
        eos_token_id = [self.tokenizer.eos_token_id] # 

        #json_datas = json_load(data_path=data_path)
        #eval_datas = data_load(data_path=data_path)

#         special_words = [
#         "#@주소#", "#@이모티콘#", "#@이름#", "#@URL#", "#@소속#",
#         "#@기타#", "#@전번#", "#@계정#", "#@url#", "#@번호#", "#@금융#", "#@신원#",
#         "#@장소#", "#@시스템#사진#", "#@시스템#동영상#", "#@시스템#기타#", "#@시스템#검색#",
#         "#@시스템#지도#", "#@시스템#삭제#", "#@시스템#파일#", "#@시스템#송금#", "#@시스템#",]
        
#         self.tokenizer.add_special_tokens({"additional_special_tokens": special_words})
        
        datas = val_load()
        
        for data in tqdm(datas[0]):
            #id = dict_data['id']
            texts = data

            tmp_str =''
            for text in texts:
                tmp_str += text

            # encode
            # truncate, if string exceed max length
            enc_tmp_str = self.tokenizer.encode(tmp_str, truncation = True, max_length=texts_max_length, add_special_tokens=False)

            #  요약할 문장  요약된 문장 
            index_of_words = bos_token_id + enc_tmp_str+ eos_token_id

            self.data.append({
                #'id':id,
                'input':torch.tensor([index_of_words]).to(device)
            })

    def __len__(self):
        return len(self.data)

    def __getitem__(self,index):
        item = self.data[index]
        return item

### def_seed부분은 아직 이해가 부족하여 수정은 하지않았습니다.

### 현재 1 epoch로 모든 train data(29만개)를 사용할 시 약 5시간이 걸립니다. 데이터를 분할하여 다양한 방법을 적용해보는 것을 목표로하고 성능 향상이 보인다면 전체 데이터를 다 사용할 예정입니다.

### 데이터 분할 기준은 각 Category에서 20%씩 추출하여 사용해볼 생각입니다.

In [13]:
# train

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from torch.utils.data import dataloader
import random
#import wandb


def set_seed(seed = 42): # 난수 생성 seed 고정
    torch.backends.cudnn.deterministic = True # Deterministic 한 알고리즘만 사용하게 하기
    torch.backends.cudnn.benchmark = False # cudnn.benchmar 활성화 여부
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
        
if __name__ == '__main__':
    
    set_seed()
    #wandb.init(project= 'bakbak', entity= 'quarter2000', name= f'KG')
    #checkpoint_path = "/aiffel/aiffel/aiffelthon/checkpoint_2/"
    #save_ckpt_path = f"{checkpoint_path}/kogpt2-abstractive_2.pth"
    checkpoint_path = "/home/rlfkr6974/Download/checkpoint"
    save_ckpt_path = f"{checkpoint_path}/kogpt2-abstractive_3epoch_sptoken_meta.pth"
    
    n_epoch = 2         # Num of Epoch
    #batch_size = 4      # 배치 사이즈
    batch_size = 8      # 배치 사이즈
    device = "cuda" if torch.cuda.is_available() else "cpu"
    #device = 'cpu'
    print('device: ',device)
    save_step = 1750 # 학습 저장 주기
    learning_rate = 5e-5  # Learning Rate

    dataset= AbstrativeDataset(device=device)
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    #checkpoint = torch.load(save_ckpt_path,map_location=torch.device(device))
    model = AbstractiveKoGPT2()
    model.to(device)
    #model.load_state_dict(checkpoint['model_state_dict'])

    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=3)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    total_losses = []
    losses =[]

    if os.path.isfile(save_ckpt_path):
        checkpoint = torch.load(save_ckpt_path, map_location=device)
        pre_epoch = checkpoint['epoch']
        pre_loss = checkpoint['loss']
        total_losses = checkpoint['losses']

        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        print(f"load pretrain from: {save_ckpt_path}, epoch={pre_epoch}, loss={pre_loss}")
        

    for epoch in tqdm(range(n_epoch)):
        count = 0
        with tqdm(total=len(train_loader), desc=f"Train({epoch})") as pbar:
            for i, data in enumerate(train_loader):
                optimizer.zero_grad()
#                 for d in data:
#                     print(max(d),min(d))
                outputs = model(data, labels=data)
                _, logits = outputs[:2]

                # Shift so that tokens < n predict n
                shift_logits = logits[..., :-1, :].contiguous()
                shift_labels = data[..., 1:].contiguous()

                loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
                loss.backward()
                optimizer.step()

                losses.append(loss.item())

                if (count > 0 and count % save_step == 0) or (len(data) < batch_size):
                    torch.save({
                        'epoch': epoch,
                        'train_no': count,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': loss,
                        'losses': losses
                    }, save_ckpt_path)
                count += 1
                pbar.update(1)
                pbar.set_postfix_str(f"Loss: {loss.item():.3f} ({np.mean(losses):.3f})")
                #if i%50==0:
                    #wandb.log({'loss':np.mean(losses)})

        total_losses.append(np.mean(losses))

#     # data
    data = {
        "loss": total_losses
    }
    df = pd.DataFrame(data)
    display(df)

#     # graph
#     plt.figure(figsize=[12, 4])
#     plt.plot(losses, label="loss")
#     plt.legend()
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.show()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


device:  cuda


100%|██████████████████████████████████████████████████████████████████| 13994/13994 [00:00<00:00, 44494.19it/s]
13994it [00:09, 1416.19it/s]


load pretrain from: /home/rlfkr6974/Download/checkpoint/kogpt2-abstractive_3epoch_sptoken_meta.pth, epoch=2, loss=3.9719858169555664


Train(0):   2%|▊                                         | 34/1750 [00:06<05:27,  5.24it/s, Loss: 3.566 (3.596)]


...

Train(1):  96%|██████████████████████████████████████▎ | 1678/1750 [05:14<00:13,  5.32it/s, Loss: 3.315 (3.449)]


Train(1):  98%|███████████████████████████████████████▏| 1713/1750 [05:21<00:06,  5.34it/s, Loss: 3.558 (3.448)]


Train(1): 100%|███████████████████████████████████████▉| 1748/1750 [05:27<00:00,  5.33it/s, Loss: 3.393 (3.447)]


100%|████████████████████████████████████████████████████████████████████████████| 2/2 [11:26<00:00, 343.46s/it]


,loss
0,6.250170
1,6.218689
2,6.007515
3,5.965816
4,5.990779
...,...
5247,3.726702
5248,3.791903
5249,3.971986
5250,3.551027


In [14]:
# test

tokenizer = get_kogpt2_tokenizer()
ckpt_path = '/home/rlfkr6974/Download/checkpoint/kogpt2-abstractive_3epoch_sptoken_meta.pth'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

eval_datas = EvalAbstrativeDataset(tokenizer=tokenizer,device=device,data_path='/home/rlfkr6974/Download/csv/val_5%.tsv')

checkpoint = torch.load(ckpt_path,map_location=torch.device(device))
model = AbstractiveKoGPT2()
model.to(device)
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

my_summaries=[]
# for data in tqdm(eval_datas[:3337]):
#     input_ids = data['input']
#     sample_output = model.generate(input_ids=input_ids,max_length=1024)
#     summary = tokenizer.decode(sample_output[0].tolist()[len(input_ids[0]):-1])
#     summary = summary.replace('','').replace('','')
#     my_summaries.append(summary)

for data in tqdm(eval_datas):
    input_ids = data['input']
    max_toc = max(input_ids[0])
    min_toc = min(input_ids[0])
    if max_toc<50000:
        sample_output = model.generate(input_ids=input_ids, max_length=256)
        summary = tokenizer.decode(sample_output[0].tolist()[len(input_ids[0]):-1], skip_special_tokens=True) 
        # 스페셜 토큰은 출력되지 않게 제한을 걸었습니다.
        summary = summary.replace('','').replace('','')
        my_summaries.append(summary)
    else:
        summary = tokenizer.decode(input_ids[0])
        summary = summary.replace('','').replace('','').replace('','')
        my_summaries.append(summary)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


cuda
cuda


100%|███████████████████████████████████████████████████████████████████████| 1746/1746 [10:24<00:00,  2.79it/s]


In [12]:
type(my_summaries)

list

In [37]:
# 요약문 dataframe으로 저장

my_summaries_df = pd.DataFrame(my_summaries)
real_sum_df = pd.DataFrame(real_Sum)

In [38]:
print(len(my_summaries_df), len(real_sum_df))

35004 35004


In [41]:
# summarization = pd.concat([my_summaries_df, real_sum_df], axis = 1)

In [45]:
# summarization.rename(columns = {0:'Model_out', 'Summary':'Real_sum'}, inplace=True)

In [47]:
# summarization.to_csv('summarization.tsv', sep = '\t',  encoding = 'utf-8', index = False)

In [13]:
# 요약 데이터 형태 확인

type(my_summaries)

list

In [14]:
len(my_summaries)

35004

In [15]:
# sum_text = pd.read_csv('/home/rlfkr6974/Download/summarization.tsv', delimiter = '\t')

In [ ]:
# 최종결과
import 
for num in range(0, ):
    print('Text : ', Input_Text[num])
    print('real_summarization : ', real_Sum[num])
    print('predict_summarization : ', my_summaries[num])
    print('-'*120)

In [19]:
import random

sum_text.sample(30)

,Model_out,Real_sum
20023,이번 주부터 단백질 셰이크를 먹는다고 한다.,이번 주부터 저녁 6시 전에 음식 섭취를 끝내고 6시 이후로는 물이나 차 종류만 마...
9320,늦게까지 안 먹어도 되는지 꼬시고 있다.,다음 날 쉬니까 늦게 가도 되는지 좀 꼬시고 있으라고 한다.
25596,다 알고 있었는데 다 알고 있었다고 한다.,기분이 슬프고 토할 것 같다고 얘기했지만 다 장난이라며 진정했다고 얘기했다.
22010,평소에는 토요일 빼고 다 있고 시험 끝나면 하루도 쉬는 날이 없다.,"평소에는 토요일 빼고 수업이 다 있고, 시험이 끝나야 쉴 수 있다."
218,오늘은 스트레스 안 받고 쉬엄쉬엄하고 있다.,오늘 일이 많고 직장동료가 외부에 회의가 있어서 외근을 나가 스트레스를 받는다.
19139,피부과에 가보려고 하는데 피부과는 보험도 안되고 피부과도 보험도 안돼서 답답하다.,피부에 트러블이 계속 나서 피부과에 가보기로 했다.
27905,엄마 아빠가 방에 들어가라고 해서 방에 들어가려고 한다.,엄마와 아빠가 방에 들어가라고 해서 들어왔지만 그 대신 자기를 보러오라고 이야기하고...
2411,오늘 원 오티 갔는데 병원마다 다르지만 오프날 교육을 한다.,나중에 병원에서 학회에 끌려가냐고 묻자 그런 것 같고 봉사와 교육도 한다는 말에 무...
32279,대동국수가 가고 싶은데 잔치국수를 좋아하지 않는다.,대동국수에서 국시를 먹기로 했다.
28095,피부과에 가야 해서 지금 머리를 하고 있다.,피부과에 가야 하는데 머리가 지금 산적 같다고 이야기한다.


In [15]:
# 실제 원문과 요약문 불러오기

Input_Text = val_load()[0]
real_Sum = val_load()[1]

# 실제 요약문 Reference로 불러오기

Reference = val_load()[1]

100%|████████████████████████████████████████████████████████████████████| 1746/1746 [00:00<00:00, 40120.17it/s]


In [ ]:
# 1 에포크 결과
# 빔서치 적용 결과

for num in range(150, 160):
    print('Text : ', Input_Text[num])
    print('real_summarization : ', real_Sum[num])
    print('predict_summarization : ', my_summaries[num])
    print('-'*120)

In [30]:
# 1 에포크 결과
# 빔서치 적용 결과

for num in range(150, 160):
    print('Text : ', Input_Text[num])
    print('real_summarization : ', real_Sum[num])
    print('predict_summarization : ', my_summaries[num])
    print('-'*120)

Text :  나도 기타 소속 다시 드간닷 낼 면접이얌 , 헐 진짜 왜 취직 안해 , 취직해야지 근데 아직 잘 모르겠어서 한 6개월 정도만 소속 서 일하게 , 구래
real_summarization :  취직을 해야 하는데 아직 몰라서 6개월 정도만 일하려고 내일 면접을 볼 거다.
predict_summarization :  아직 잘 모르겠어서 6개월 정도만 일을 하게 되었다.
------------------------------------------------------------------------------------------------------------------------
Text :  조금 늦게 되면 조퇴쓰겠다고 연락드렸는데 회사에 있기시러서 정밀 끝나고 4시에 퇴근하려구요 , 그려요 하고 싶은거 다 해요 팀장님 그 정도는 이해해주시겠죠 양심이 있으면 , 휴가가 없어서 이렇게 쓰면 안되는뎅 , 일단오늘만 사는걸로 , 그르게요 , 내일 당장 퇴사할지 어케 알아요 , 맞아요 , 일단 목표는 대출때까지임 그 이후엔 프리
real_summarization :  휴가는 없고 회사에 있기 싫어서 4시에 퇴근하려고 한다.
predict_summarization :  오늘만 사는 걸로 목표를 정했다.
------------------------------------------------------------------------------------------------------------------------
Text :  이름 어찌 됐어 , 아까 점심에 말햇는데 뭔가 뜨뜨미지근해 긍데 그사람이 우리 랩사람이 아니라서 그냥 냅둿엉 다른랩인데 왕따라서 울방에 잇는거라 좀 성격이 지랄맞아서 저번에도 나 연락처 안주긴 햇어 , 아하 글쿤 그럼 됐어 , 웅웅 혹시 말하면 연락주껭 좀 조증이 잇는 사람이라 , 앙앙 알겠엉 나도 갑자기 생각나스
real_summarization :  직장 내 성격이 맞지 않는 사람에 대해 얘기하고 있다.
predict_summa

In [ ]:
# 1 에포크 결과
# 샘플링 적용 결과

for num in range(150, 160):
    print('Text : ', Input_Text[num])
    print('real_summarization : ', real_Sum[num])
    print('predict_summarization : ', my_summaries[num])
    print('-'*120)

In [28]:
# 1 에포크 결과
# 빔서치 적용
# 메타데이터 적용

for num in range(34000, 35000, 100):
    print('Text : ', Input_Text[num])
    print('real_summarization : ', real_Sum[num])
    print('predict_summarization : ', my_summaries[num])
    print('-'*120)

Text :  #식음료# 씨그램이라는 탄산수에요 3가지맛이 있는데 저는 라임맛을 제일 좋아하죠 , 그거 코카콜라꺼 였네요 , 네.맞아요 , 엘지가 인수했잖아요 코카콜라 , 그건 몰랐던 사실인데요. 정말이에요? , 네 꽤 오래되었는데 잘모르시더라고요 2007년도에 인수했어요
real_summarization :  씨그램 탄산수 중에 라임 맛을 좋아하고 코카콜라 회사인데 엘지가 코카콜라를 인수했다고 한다.
predict_summarization :  씨그램이라는 탄산수에 라임맛을 좋아해서 코카콜라를 인수했다고 한다.
------------------------------------------------------------------------------------------------------------------------
Text :  #식음료# 수박 남았니 우리집에 너무 넘쳐나는데 다 먹어가면 사지말고 있어주겠니 잘라갈겡 토요일에! , 생각보다 금방 안먹어져 , 아니 한통 겨우 다먹고 이제 한통 잘라는데 엄마가 반통남았었다고 하고 아빠가 화천놀러갔다가 안먹고 수박 그대로 다시 가져왔엉 브엥 , 그래 가쟈왘
real_summarization :  집에 수박이 많이 남아 토요일에 잘라가기로 했다.
predict_summarization :  수박 한 통을 다 먹어가면 사지 말고 있으라고 한다.
------------------------------------------------------------------------------------------------------------------------
Text :  #식음료# 뭘먹을지 정한다면 밥버거 각이다 , 오 , 밥버거 먹고 오랫만에 , 키 그래 젱 가까운둣 , 아니면 오랫만에 순두부 , , 찌개먹어도 좋아 왜야면 오늘 현금엤음 쓰봉산다고 돈 뽑음
real_summarization :  밥버거 또는 순두부찌개를 먹자고 한다.
predict_summarization :  밥버거를 먹고 오랫만에 순두부

### rouge.get_scores 사용 방법

- rouge.get_scores(Model이 요약한 문장, 실제 요약문, avg = True)

In [16]:
# # Rouge-Score 출력 - 5epoch - 이거 실행
# # 전처리 + 스페셜 토큰 + 메타 데이터
# # 검증용 데이터 5%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

{'rouge-1': {'r': 0.154952506000734,
  'p': 0.2171953427560226,
  'f': 0.17389830428310357},
 'rouge-2': {'r': 0.04824888726349892,
  'p': 0.06967167356451681,
  'f': 0.054308264132210356},
 'rouge-l': {'r': 0.14970734525064214,
  'p': 0.21033320511639353,
  'f': 0.16813751101915153}}

In [12]:
# # Rouge-Score 출력 - 3epoch
# # 전처리 + 스페셜 토큰 + 메타 데이터
# # 검증용 데이터 5%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

{'rouge-1': {'r': 0.16409849666847268,
  'p': 0.22783248959014696,
  'f': 0.18354316621133804},
 'rouge-2': {'r': 0.05482481208846422,
  'p': 0.07738087131186899,
  'f': 0.0614405817281854},
 'rouge-l': {'r': 0.15680600706978745,
  'p': 0.218139914576085,
  'f': 0.17553811923787724}}

In [21]:
# # Rouge-Score 출력 - 1epoch
# # 전처리 + 스페셜 토큰 + 메타 데이터
# # 검증용 데이터 5%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

{'rouge-1': {'r': 0.16098925834228192,
  'p': 0.20419934218681224,
  'f': 0.17265401154591506},
 'rouge-2': {'r': 0.04879227630592049,
  'p': 0.05994781239659861,
  'f': 0.05138004666611969},
 'rouge-l': {'r': 0.15429746597349767,
  'p': 0.1965380922840525,
  'f': 0.16571121128888963}}

In [12]:
# Rouge-Score 출력 - 1epoch 
# 전처리 + 스페셜 토큰
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

{'rouge-1': {'r': 0.15961884247232697,
  'p': 0.2014298580961297,
  'f': 0.1710888949585144},
 'rouge-2': {'r': 0.049620977272450885,
  'p': 0.06076377831229271,
  'f': 0.05213995795399614},
 'rouge-l': {'r': 0.15328992718749213,
  'p': 0.19392421145818078,
  'f': 0.16443930346830654}}

In [11]:
# Rouge-Score 출력 - 1epoch
# Normal
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)


{'rouge-1': {'r': 0.14604635980906974,
  'p': 0.16112444107272925,
  'f': 0.14016515684045291},
 'rouge-2': {'r': 0.04056878558861431,
  'p': 0.04525382401247486,
  'f': 0.039325431776571225},
 'rouge-l': {'r': 0.14048426605935824,
  'p': 0.1559405899340719,
  'f': 0.13536457122560458}}

In [22]:
# Rouge-Score 출력 - 1epoch
# num_beams = 5 적용
# num_return_sequences = 3 적용
# length_penalty = 1.2 적용
# 검증용 데이터 100%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)


{'rouge-1': {'r': 0.19438815082812133,
  'p': 0.2639137046036113,
  'f': 0.21574304435661867},
 'rouge-2': {'r': 0.073385078848413,
  'p': 0.1025911356762314,
  'f': 0.08192278835309064},
 'rouge-l': {'r': 0.18730025451857873,
  'p': 0.25430491871634575,
  'f': 0.20787998119252368}}

In [24]:
# Rouge-Score 출력 - 1epoch
# num_beams = 5 적용
# num_return_sequences = 3 적용
# length_penalty = 1.2 적용
# 메타데이터 적용 (스페셜토큰에 포함)
# 검증용 데이터 100%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

{'rouge-1': {'r': 0.19510236546996204,
  'p': 0.2635285375270567,
  'f': 0.21607333818741667},
 'rouge-2': {'r': 0.07368468199436151,
  'p': 0.10216011840326177,
  'f': 0.08201022752804889},
 'rouge-l': {'r': 0.18789082041969174,
  'p': 0.2538517430154955,
  'f': 0.20811449011011465}}

In [14]:
# Rouge-Score 출력 - 1epoch
# num_beams = 5 적용
# num_return_sequences = 3 적용
# length_penalty = 1.2 적용
# 메타데이터 미적용 
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

{'rouge-1': {'r': 0.15294172120107125,
  'p': 0.2320318646981268,
  'f': 0.17736299194554428},
 'rouge-2': {'r': 0.05269916920612938,
  'p': 0.08013172633017994,
  'f': 0.06069564462408859},
 'rouge-l': {'r': 0.1487336761345979,
  'p': 0.22607683065536288,
  'f': 0.1726238771855908}}

In [14]:
# Rouge-Score 출력 - 3epoch
# num_beams = 5 적용
# num_return_sequences = 3 적용
# length_penalty = 1.2 적용
# 검증용 데이터 100%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)


{'rouge-1': {'r': 0.18639885707671078,
  'p': 0.19329617359854764,
  'f': 0.1735222100186861},
 'rouge-2': {'r': 0.0635039570135198,
  'p': 0.07400411871047714,
  'f': 0.06281411761127186},
 'rouge-l': {'r': 0.17725419727268382,
  'p': 0.1850924510870746,
  'f': 0.16576700380589335}}

In [12]:
# Rouge-Score 출력 - 1epoch
# 
#do_sample=True,
# length_penalty = 1.2 적용
# top_p=0.92,
# top_k=50,
# temperature= 0.8,
# num_return_sequences = 3 적용

# 검증용 데이터 100%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

{'rouge-1': {'r': 0.1675306963882514,
  'p': 0.1803165341796574,
  'f': 0.16639589547652903},
 'rouge-2': {'r': 0.04712794044526311,
  'p': 0.04989784937911874,
  'f': 0.04602992427397577},
 'rouge-l': {'r': 0.15859230211282327,
  'p': 0.1708759484728981,
  'f': 0.15756247056193307}}